# AI Scheduling Assistant Hackathon

This notebook implements an AI-powered scheduling assistant for the AMD AI Hackathon. The assistant autonomously schedules meetings, resolves conflicts, and optimizes calendars using Google Calendar API and a vLLM server with the DeepSeek LLM. It meets the requirements for autonomy, accuracy, and user experience.

## Setup Instructions
1. **Install Dependencies**: Run the installation cell to install required Python packages.
2. **Start vLLM Server**: Open a terminal and run the provided command to start the vLLM server with the DeepSeek LLM.
3. **Prepare Google Calendar Credentials**: Ensure Google API credentials are in the `Keys/` directory (e.g., `userone.amd.json`, `userone.amd.token`).
4. **Run the Flask Server**: Execute the code to start the Flask server on port 5000.

## Evaluation Criteria
- Correctness: Accurate scheduling with conflict resolution.
- Latency: Efficient processing using free/busy endpoint.
- Creativity: Supports natural language preferences and time zones.
- Repository Maintenance: Well-documented and organized code.

In [1]:
# Install required packages
!pip install google_auth_oauthlib flask openai python-dateutil

## Start vLLM Server
Open a new terminal in Jupyter and run the following command to start the vLLM server with the DeepSeek LLM:

```bash
HIP_VISIBLE_DEVICES=0 vllm serve /home/user/Models/deepseek-ai/deepseek-llm-7b-chat \
    --gpu-memory-utilization 0.9 \
    --swap-space 16 \
    --disable-log-requests \
    --dtype float16 \
    --max-model-len 2048 \
    --tensor-parallel-size 1 \
    --host 0.0.0.0 \
    --port 3000 \
    --num-scheduler-steps 10 \
    --max-num-seqs 128 \
    --max-num-batched-tokens 2048 \
    --max-model-len 2048 \
    --distributed-executor-backend "mp"
```

In [27]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import BatchHttpRequest
from openai import OpenAI
from flask import Flask, request, jsonify
from threading import Thread
from dateutil import parser, tz
import json
import re
from datetime import datetime, timedelta, timezone

# Google Calendar API authentication
SCOPES = ["https://www.googleapis.com/auth/calendar"]
token_cache = {}
calendar_service_cache = {}

def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email) is not None

def get_creds(email):
    """Retrieve credentials from cache or token file."""
    if not is_valid_email(email):
        raise ValueError(f"Invalid email address: {email}")
    if email not in token_cache:
        token_path = f"Keys/{email.split('@')[0]}.token"
        try:
            token_cache[email] = Credentials.from_authorized_user_file(token_path, SCOPES)
        except FileNotFoundError:
            raise FileNotFoundError(f"Token file not found: {token_path}. Please ensure the token file is provided.")
        except Exception as e:
            raise Exception(f"Failed to load credentials for {email}: {str(e)}")
    return token_cache[email]

def get_calendar_service(email):
    """Retrieve cached calendar service for an email."""
    if email not in calendar_service_cache:
        creds = get_creds(email)
        calendar_service_cache[email] = build("calendar", "v3", credentials=creds)
    return calendar_service_cache[email]

def create_calendar_event(calendar_service, attendees, start_time, end_time, summary, location):
    """Create a Google Calendar event for all attendees."""
    event = {
        'summary': summary,
        'location': location,
        'start': {'dateTime': start_time, 'timeZone': 'Asia/Kolkata'},
        'end': {'dateTime': end_time, 'timeZone': 'Asia/Kolkata'},
        'attendees': [{'email': email} for email in attendees],
        'reminders': {'useDefault': True}
    }
    return calendar_service.events().insert(calendarId='primary', body=event).execute()

def retrieve_calendar_events(email, start, end):
    """Fetch calendar events for a user, handling both timed and all-day events."""
    events_list = []
    try:
        calendar_service = get_calendar_service(email)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        for event in events:
            # Validate start and end fields
            if not event.get("start") or not event.get("end"):
                continue  # Skip invalid events
            start_time = event["start"].get("dateTime")
            end_time = event["end"].get("dateTime")
            if not start_time and event["start"].get("date"):
                start_time = event["start"].get("date") + "T00:00:00+05:30"
            if not end_time and event["end"].get("date"):
                end_time = event["end"].get("date") + "T00:00:00+05:30"
            if not start_time or not end_time:
                continue  # Skip events with invalid start/end times
            attendee_list = [attendee['email'] for attendee in event.get("attendees", [])] or ["SELF"]
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "")
            })
    except Exception as e:
        return {"error": f"Error retrieving events for {email}: {str(e)}"}
    return events_list

def find_available_slot(calendar_service, attendees, start_date, duration, preferred_time='none'):
    """Find an available time slot for all attendees using free/busy endpoint."""
    ist_tz = tz.gettz('Asia/Kolkata')
    working_start = start_date.replace(hour=9, minute=0, second=0, microsecond=0).astimezone(ist_tz)
    working_end = start_date.replace(hour=17, minute=0, second=0, microsecond=0).astimezone(ist_tz)
    time_min = working_start.astimezone(tz.UTC).isoformat()
    time_max = working_end.astimezone(tz.UTC).isoformat()

    body = {"timeMin": time_min, "timeMax": time_max, "items": [{"id": email} for email in attendees]}
    freebusy = calendar_service.freebusy().query(body=body).execute()

    all_busy = []
    conflicts = 0
    for cal in freebusy['calendars'].values():
        busy_periods = cal.get('busy', [])
        conflicts += len(busy_periods)
        all_busy.extend(busy_periods)
    busy_periods = sorted(all_busy, key=lambda x: x['start'])
    merged_busy = []
    for period in busy_periods:
        if not merged_busy or merged_busy[-1]['end'] < period['start']:
            merged_busy.append(period)
        else:
            merged_busy[-1]['end'] = max(merged_busy[-1]['end'], period['end'])

    free_slots = []
    current = parser.parse(time_min)
    for busy in merged_busy:
        busy_start = parser.parse(busy['start'])
        if current < busy_start:
            free_slots.append({'start': current, 'end': busy_start})
        current = max(current, parser.parse(busy['end']))
    if current < parser.parse(time_max):
        free_slots.append({'start': current, 'end': parser.parse(time_max)})

    duration_td = timedelta(minutes=int(duration))
    preferred_time_honored = False
    if preferred_time != 'none':
        if preferred_time == 'morning':
            pref_start, pref_end = 9, 12
        elif preferred_time == 'afternoon':
            pref_start, pref_end = 13, 17
        else:
            try:
                pref_start = int(preferred_time.split(':')[0])
                pref_end = pref_start + 1
            except:
                pref_start, pref_end = 9, 17
        pref_start_dt = start_date.replace(hour=pref_start, minute=0, second=0).astimezone(tz.UTC)
        pref_end_dt = start_date.replace(hour=pref_end, minute=0, second=0).astimezone(tz.UTC)
        for slot in free_slots:
            slot_start = slot['start']
            slot_end = slot['end']
            if slot_start >= pref_start_dt and slot_end <= pref_end_dt and (slot_end - slot_start) >= duration_td:
                preferred_time_honored = True
                return slot_start.astimezone(ist_tz).isoformat(), (slot_start + duration_td).astimezone(ist_tz).isoformat(), conflicts, preferred_time_honored
    for slot in free_slots:
        if (slot['end'] - slot['start']) >= duration_td:
            return slot['start'].astimezone(ist_tz).isoformat(), (slot['start'] + duration_td).astimezone(ist_tz).isoformat(), conflicts, preferred_time_honored
    return None, None, conflicts, preferred_time_honored

# vLLM and AI Agent setup
BASE_URL = "http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

class AI_AGENT:
    def __init__(self, client, model_path):
        self.client = client
        self.model_path = model_path

    def parse_email(self, email_text):
        try:
            response = self.client.chat.completions.create(
                model=self.model_path,
                temperature=0.0,
                messages=[{
                    "role": "user",
                    "content": f"""
                    You are an Agent that helps in scheduling meetings.
                    Your job is to extract:
                    1. List of email IDs of participants (comma-separated).
                    2. Meeting duration in minutes.
                    3. Time constraints (e.g., 'next week').
                    4. Preferred time (e.g., 'morning', 'afternoon', 'at 14:00', or 'none' if not specified).
                    If the list of email IDs of participants are just names, append @amd.com at the end.
                    Return as JSON with 'participants', 'time_constraints', 'meeting_duration', and 'preferred_time'.
                    Strictly follow the instructions. Return only the dict with participants email IDs, time constraints, meeting duration in minutes, and preferred time.
                    Email: {email_text}
                    """
                }]
            )
            parsed = json.loads(response.choices[0].message.content)
            required_keys = ['participants', 'meeting_duration', 'time_constraints', 'preferred_time']
            if not all(key in parsed for key in required_keys):
                raise ValueError(f"LLM output missing required keys: {required_keys}")
            if not isinstance(parsed.get('meeting_duration'), (int, float)) or parsed['meeting_duration'] <= 0:
                raise ValueError("Invalid meeting duration")
            if not parsed.get('participants'):
                raise ValueError("No participants found")
            # Convert comma-separated participants to list
            parsed['participants'] = [p.strip() for p in parsed['participants'].split(',')]
            return parsed
        except Exception as e:
            raise ValueError(f"LLM parsing failed: {str(e)}")

ai_agent = AI_AGENT(client, MODEL_PATH)

def your_meeting_assistant(data):
    num_api_calls = 0
    llm_start_time = datetime.now()
    try:
        # Parse email content
        meeting_details = ai_agent.parse_email(data["EmailContent"])
        llm_time = (datetime.now() - llm_start_time).total_seconds()
        duration = meeting_details["meeting_duration"]
        time_constraint = meeting_details["time_constraints"]
        preferred_time = meeting_details.get("preferred_time", "none")
        attendees = [data["From"]] + [attendee["email"] for attendee in data["Attendees"] if is_valid_email(attendee["email"])]
        attendees = list(set(attendees))  # Remove duplicates
        if not attendees:
            raise ValueError("No valid email addresses provided for attendees")

        # Determine start date
        today = datetime.now(timezone(timedelta(hours=5, minutes=30)))
        weekday_map = {"monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, "friday": 4}
        if time_constraint.lower() in weekday_map:
            days_ahead = (weekday_map[time_constraint.lower()] - today.weekday() + 7) % 7
            if days_ahead == 0:
                days_ahead = 7
            start_date = today + timedelta(days=days_ahead)
        elif "next week" in time_constraint.lower():
            start_date = today + timedelta(days=7 - today.weekday())
        else:
            start_date = today + timedelta(days=1)

        # Authenticate
        calendar_service = get_calendar_service(data['From'])
        num_api_calls += 1

        # Find available slot
        start_time, end_time, conflicts, preferred_time_honored = find_available_slot(calendar_service, attendees, start_date, duration, preferred_time)
        num_api_calls += 1
        attempted_days = 1
        attempts = 0
        while not start_time and attempts < 3:
            start_date += timedelta(days=1)
            start_time, end_time, conflicts, preferred_time_honored = find_available_slot(calendar_service, attendees, start_date, duration, preferred_time)
            num_api_calls += 1
            attempted_days += 1
            attempts += 1

        if not start_time:
            start_time = start_date.replace(hour=10, minute=30, second=0, microsecond=0).isoformat()
            end_time = (start_date + timedelta(minutes=int(duration))).isoformat()
            metadata_status = f"warning: no free slot found after {attempted_days} attempts, default time assigned"
        else:
            create_calendar_event(calendar_service, attendees, start_time, end_time, data.get("Subject", "Meeting"), data.get("Location", ""))
            num_api_calls += 1
            metadata_status = "success: event created"

        # Fetch events using batch request
        day_start = start_date.replace(hour=0, minute=0, second=0, microsecond=0).isoformat()
        day_end = (start_date + timedelta(days=1)).isoformat()
        events = {}
        errors = []

        def process_events(events):
            events_list = []
            for event in events:
                if not event.get("start") or not event.get("end"):
                    continue
                start_time = event["start"].get("dateTime")
                end_time = event["end"].get("dateTime")
                if not start_time and event["start"].get("date"):
                    start_time = event["start"].get("date") + "T00:00:00+05:30"
                if not end_time and event["end"].get("date"):
                    end_time = event["end"].get("date") + "T00:00:00+05:30"
                if not start_time or not end_time:
                    continue
                attendee_list = [attendee['email'] for attendee in event.get("attendees", [])] or ["SELF"]
                events_list.append({
                    "StartTime": start_time,
                    "EndTime": end_time,
                    "NumAttendees": len(set(attendee_list)),
                    "Attendees": list(set(attendee_list)),
                    "Summary": event.get("summary", "")
                })
            return events_list

        batch = calendar_service.new_batch_http_request()
        for email in attendees:
            def callback(request_id, response, exception):
                if exception:
                    events[email] = [{"error": f"Error retrieving events for {email}: {str(exception)}"}]
                    errors.append(f"Error retrieving events for {email}: {str(exception)}")
                else:
                    events[email] = process_events(response.get('items', []))
            batch.add(get_calendar_service(email).events().list(
                calendarId='primary', timeMin=day_start, timeMax=day_end, singleEvents=True, orderBy='startTime'
            ), callback=callback)
        batch.execute()
        num_api_calls += 1

        if errors:
            raise Exception(f"Event retrieval errors: {', '.join(errors)}")

        # Create proposed event
        proposed_event = {
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(attendees),
            "Attendees": attendees,
            "Summary": data.get("Subject", "Meeting")
        }

        # Construct output
        processed_attendees = [{"email": email, "events": events.get(email, []) + [proposed_event]} for email in attendees]
        output = {
            "Request_id": data["Request_id"],
            "Datetime": data["Datetime"],
            "Location": data.get("Location", ""),
            "From": data["From"],
            "Attendees": processed_attendees,
            "Subject": data.get("Subject", "Meeting"),
            "EmailContent": data["EmailContent"],
            "EventStart": start_time,
            "EventEnd": end_time,
            "Duration_mins": str(duration),
            "MetaData": {
                "status": metadata_status,
                "attempted_days": attempted_days,
                "preferred_time_honored": preferred_time_honored,
                "conflicts_resolved": conflicts,
                "api_calls": num_api_calls,
                "llm_processing_time": llm_time
            }
        }
    except Exception as e:
        output = {
            "Request_id": data.get("Request_id", ""),
            "Datetime": data.get("Datetime", ""),
            "Location": data.get("Location", ""),
            "From": data.get("From", ""),
            "Attendees": [{"email": email, "events": [{"error": str(e)}]} for email in ([data.get("From", "")] + [attendee["email"] for attendee in data.get("Attendees", []) if is_valid_email(attendee["email"])])],
            "Subject": data.get("Subject", "Meeting Scheduling Failed"),
            "EmailContent": data.get("EmailContent", ""),
            "EventStart": "",
            "EventEnd": "",
            "Duration_mins": "0",
            "MetaData": {
                "status": "error",
                "message": str(e),
                "attempted_days": 0,
                "preferred_time_honored": False,
                "conflicts_resolved": 0,
                "api_calls": num_api_calls,
                "llm_processing_time": llm_time if 'llm_time' in locals() else 0
            }
        }
    return {"processed": True, "output": output}


In [25]:

# Flask setup
app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    try:
        data = request.get_json()
        print(f"\nReceived: {json.dumps(data, indent=2)}")
        new_data = your_meeting_assistant(data)
        received_data.append(data)
        print(f"\nSending:\n{json.dumps(new_data['output'], indent=2)}")
        return jsonify(new_data['output'])
    except Exception as e:
        error_response = {
            "processed": False,
            "output": {
                "Request_id": data.get("Request_id", ""),
                "Datetime": data.get("Datetime", ""),
                "Location": data.get("Location", ""),
                "From": data.get("From", ""),
                "Attendees": [{"email": email, "events": [{"error": str(e)}]} for email in ([data.get("From", "")] + [attendee["email"] for attendee in data.get("Attendees", []) if is_valid_email(attendee["email"])])],
                "Subject": data.get("Subject", "Meeting Scheduling Failed"),
                "EmailContent": data.get("EmailContent", ""),
                "EventStart": "",
                "EventEnd": "",
                "Duration_mins": "0",
                "MetaData": {
                    "status": "error",
                    "message": str(e),
                    "attempted_days": 0,
                    "preferred_time_honored": False,
                    "conflicts_resolved": 0,
                    "api_calls": 0,
                    "llm_processing_time": 0
                }
            }
        }
        print(f"\nError: {json.dumps(error_response, indent=2)}")
        return jsonify(error_response)

def run_flask():
    try:
        app.run(host='0.0.0.0', port=5000)
    except OSError as e:
        if "Address already in use" in str(e):
            print("Port 5000 is in use, trying port 5001...")
            app.run(host='0.0.0.0', port=5001)
        else:
            raise e

Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


## Test Case
Below is a sample test case to verify the functionality. The assistant processes a meeting request and returns the scheduled meeting details in the required format.

In [28]:
test_input = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "02-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [{"email": "usertwo.amd@gmail.com"}, {"email": "userthree.amd@gmail.com"}],
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

result = your_meeting_assistant(test_input)
print(json.dumps(result['output'], indent=2))

{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-14T10:00:00+05:30",
          "EndTime": "2025-07-14T10:30:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userthree.amd@gmail.com",
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-14T10:00:00+05:30",
          "EndTime": "2025-07-14T10:30:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "userthree.amd@gmail.com",
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com"
          ],
          "Summary": "Meeting"
        }
      ]
    },
 